In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler,RobustScaler,Normalizer
from sklearn.model_selection import KFold

In [2]:
import csv
col_names=['id','Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape','Marginial Adhesion','Single Epithelial Cell Size','Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class']
f=pd.read_csv('breast-cancer-wisconsin.data',header=None,names=col_names)
f

,id,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginial Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [3]:
MF=f.drop_duplicates() 
MF=MF.drop(columns=['id'])
#TODO: check if there could are over or lower than 1~10 int float value.
#TODO: check if there are string value


# drop '?' in Bare Nucile and convert to float
MF.loc[MF['Bare Nuclei']=="?","Bare Nuclei"]= np.nan 
MF.dropna(inplace=True,axis=0) 
MF=MF.astype('float')

In [4]:
X=MF[['Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape','Marginial Adhesion','Single Epithelial Cell Size','Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses']]
y=np.array(pd.DataFrame(MF,columns=['Class']))


In [5]:
def auto_scaling(df)->list:
    scaling_result=[]
    scaling_result.append(StandardScaler().fit_transform(df))
    scaling_result.append(MinMaxScaler().fit_transform(df))
    scaling_result.append(MaxAbsScaler().fit_transform(df))
    scaling_result.append(RobustScaler().fit_transform(df))
    scaling_result.append(Normalizer().fit_transform(df))
    return scaling_result

In [6]:
scaled_X=auto_scaling(X)

In [7]:
def auto_algorithm(scaled_X,y):
    DCT_EN={"name":"DecisionTreeClassifier_Entropy","score":0,"scale":0,"n":0}
    DCT_GINI={"name":"DecisionTreeClassifier_Gini","score":0,"scale":0}
    LR={"name":"Logisic Regression","score":0,"scale":0}
    SVM={"name":"Support Vector Machine","score":0,"scale":0}
  
    for n in range (3,12):
       
        kf = KFold(n_splits=n, shuffle=True, random_state=1)
        for i in range(0,5):
            X_train, X_test, y_train, y_test = train_test_split(scaled_X[i], y, test_size=0.2, shuffle=True, random_state=1)
            dtc=DecisionTreeClassifier(criterion='entropy')
            dtc.fit(X_train,y_train)
            
            if DCT_EN["score"]<dtc.score(X_test,y_test):
                DCT_EN["score"]=dtc.score(X_test,y_test)
                DCT_EN["scale"]=i
                DCT_EN["n"]=n    
        for i in range(0,5):
            X_train, X_test, y_train, y_test = train_test_split(scaled_X[i], y, test_size=0.2, shuffle=True, random_state=1)
            dtc=DecisionTreeClassifier(criterion='gini')
            dtc.fit(X_train,y_train)
           
            if DCT_GINI["score"]<dtc.score(X_test,y_test):
                DCT_GINI["score"]=dtc.score(X_test,y_test)
                DCT_GINI["scale"]=i
                DCT_GINI["n"]=n        
    
        for i in range(0,5):
            lr=LogisticRegression()
            lr.fit(X_train,y_train)
            pred=lr.predict(X_test)
            if LR["score"]<lr.score(X_test,y_test):
                LR["score"]=lr.score(X_test,y_test)
                LR["scale"]=i
                LR["n"]=n   
    
        for i in range(0,5):
            SM=svm.SVR()
            SM.fit(X_train,y_train)
            pred=SM.predict(X_test)
            if SVM["score"]<SM.score(X_test,y_test):
                SVM["score"]=SM.score(X_test,y_test)
                SVM["scale"]=i
                SVM["n"]=n   
    return DCT_EN,DCT_GINI,LR,SVM

In [8]:
print("the Number of scale 0:StandardScaler, 1:MinMaxScaler, 2:MaxabsScaler, 3:RobustScaler, 4:Normalizer")
print("the n means the proper n of K-fold")
auto_algorithm(scaled_X,y)

the Number of scale 0:StandardScaler, 1:MinMaxScaler, 2:MaxabsScaler, 3:RobustScaler, 4:Normalizer
the n means the proper n of K-fold


c:\Users\aenor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\aenor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\aenor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\aenor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

({'name': 'DecisionTreeClassifier_Entropy',
  'score': 0.9407407407407408,
  'scale': 2,
  'n': 3},
 {'name': 'DecisionTreeClassifier_Gini',
  'score': 0.9481481481481482,
  'scale': 1,
  'n': 3},
 {'name': 'Logisic Regression',
  'score': 0.8592592592592593,
  'scale': 0,
  'n': 3},
 {'name': 'Support Vector Machine',
  'score': 0.6871782088653948,
  'scale': 0,
  'n': 3})